In [1]:
from importlib import reload
import sys
import util
reload(util)
data = util.FontData
FontDataset = util.FontDataset
import experiment
reload(experiment)
from sklearn import linear_model, neural_network

In [2]:
data.load()

Loading embeddings...done
Loading typographic + semantic vectors...done
Loading images...done
Loading glyphs...done
Loading SVGs...done


In [3]:
train_dataset = FontDataset(data, 'train')
val_dataset = FontDataset(data, 'val')
test_dataset = FontDataset(data, 'test')


In [16]:
train_typographic = data.get_all_typographic('train')
val_typographic = data.get_all_typographic('val')
test_typographic = data.get_all_typographic('test')

In [26]:
train_image = data.get_all_image('train')
train_image = train_image.reshape(train_image.shape[0], -1)
val_image = data.get_all_image('val')
val_image = val_image.reshape(val_image.shape[0], -1)
test_image = data.get_all_image('test')
test_image = test_image.reshape(test_image.shape[0], -1)

In [27]:
train_semantic = data.get_all_semantic('train')
val_semantic = data.get_all_semantic('val')
test_semantic = data.get_all_semantic('test')

### Linear Regression: Image -> Typographic

In [28]:
image_typographic_reg = linear_model.LinearRegression()

In [29]:
image_typographic_reg.fit(train_image, train_typographic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [30]:
train_stats, train_typographic_pred = experiment.run_experiment(image_typographic_reg, train_image, train_typographic)

{'RMSE': 1.3977822845845367e-06, 'R2': 0.9999999998341788}


In [31]:
val_stats, val_typographic_pred = experiment.run_experiment(image_typographic_reg, val_image, val_typographic)

{'RMSE': 0.10177131742238998, 'R2': 0.5893038248632995}


In [32]:
test_stats, test_typographic_pred = experiment.run_experiment(image_typographic_reg, test_image, test_typographic)

{'RMSE': 0.07531864196062088, 'R2': 0.4952896890912051}


### Linear Regression: Typographic -> Semantic

In [33]:
typographic_semantic_reg = linear_model.LinearRegression()

In [34]:
typographic_semantic_reg.fit(train_typographic, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [35]:
train_stats, train_semantic_pred = experiment.run_experiment(typographic_semantic_reg, train_typographic, train_semantic)

{'RMSE': 0.10326619446277618, 'R2': 0.5034361427743564}


In [36]:
val_stats, val_semantic_pred = experiment.run_experiment(typographic_semantic_reg, val_typographic, val_semantic)

{'RMSE': 0.106668621301651, 'R2': 0.5101831289972383}


In [46]:
test_stats, test_semantic_pred = experiment.run_experiment(typographic_semantic_reg, test_typographic, test_semantic)

{'RMSE': 0.1053079217672348, 'R2': 0.4863880163141386}


### Linear Regressions: Image -> Typographic -> Semantic

In [37]:
class SequentialLinearRegression:
    def __init__(self, reg1, reg2):
        self.reg1 = reg1
        self.reg2 = reg2
    def predict(self, X):
        return self.reg2.predict(self.reg1.predict(X))

image_typographic_semantic_reg = SequentialLinearRegression(image_typographic_reg, typographic_semantic_reg)

In [38]:
train_semantic

array([[0.33920993, 0.24672677, 0.45607575, ..., 0.46512891, 0.59643258,
        0.54990847],
       [0.49424582, 0.46839964, 0.37099306, ..., 0.65840484, 0.43551042,
        0.48553453],
       [0.24767428, 0.07544883, 0.25545198, ..., 0.65944604, 0.37622461,
        0.45430069],
       ...,
       [0.73453555, 0.70904671, 0.56249398, ..., 0.47190857, 0.76819777,
        0.45068339],
       [0.74848685, 0.62451316, 0.45640533, ..., 0.50166912, 0.41377995,
        0.35790375],
       [0.71824549, 0.84435252, 0.47846678, ..., 0.72807055, 0.45456135,
        0.31400816]])

In [39]:
train_stats, train_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, train_image, train_semantic)

{'RMSE': 0.1032661497592926, 'R2': 0.503436530931538}


In [40]:
val_stats, val_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, val_image, val_semantic)

{'RMSE': 0.11760729551315308, 'R2': 0.4122227357775069}


In [47]:
test_stats, test_semantic_pred = experiment.run_experiment(typographic_semantic_reg, test_typographic, test_semantic)

{'RMSE': 0.1053079217672348, 'R2': 0.4863880163141386}


### Linear Regression: Image -> Semantic

In [41]:
image_semantic_reg = linear_model.LinearRegression()

In [42]:
image_semantic_reg.fit(train_image, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [43]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_reg, train_image, train_semantic)

{'RMSE': 1.3475322475642315e-06, 'R2': 0.9999999999189795}


In [44]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_reg, val_image, val_semantic)

{'RMSE': 0.07973702996969223, 'R2': 0.7187900568850076}


In [48]:
test_stats, test_semantic_pred = experiment.run_experiment(image_semantic_reg, test_image, test_semantic)

{'RMSE': 0.06820797920227051, 'R2': 0.7730728338637595}


### Multilayer Perceptron: Image -> Semantic

In [28]:
image_semantic_nn = neural_network.MLPRegressor(hidden_layer_sizes=(10,), max_iter=500)

In [29]:
image_semantic_nn.fit(train_image, train_semantic)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=500, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [30]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_nn, train_image, train_semantic)

{'RMSE': 0.16850465536117554, 'R2': -0.674681103926215}


In [31]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_nn, val_image, val_semantic)

{'RMSE': 0.17422597110271454, 'R2': -0.46043920166494035}


#### Example